# Calculation of municipal area and density of agricultural and livestock farms (Spain, 2020)


## 1. Import required libraries


In [ ]:
# Basic QGIS libraries
from qgis.core import *
from PyQt5.QtCore import QVariant
import processing

In [ ]:
# Verification
# List ALL layer names loaded in the project
for lyr in QgsProject.instance().mapLayers().values():
    print(lyr.name())

## 2. Load original layers from the project

In [ ]:
# Agricultural farms layer
agr_layer = QgsProject.instance().mapLayersByName("NumExpAgr_2020")[0]

# Livestock farms layer
gan_layer = QgsProject.instance().mapLayersByName("NumExpGan_2020")[0]


## 3. Reproject layers to a CRS in meters

In [ ]:
# Reproject the AGRICULTURAL farms layer to ETRS89 / UTM 30N
agr_utm = processing.run(
    "native:reprojectlayer",
    {
        'INPUT': agr_layer,
        'TARGET_CRS': QgsCoordinateReferenceSystem('EPSG:25830'),
        'OUTPUT': 'memory:agr_utm'
    }
)['OUTPUT']

# Reproject the LIVESTOCK farms layer to ETRS89 / UTM 30N
gan_utm = processing.run(
    "native:reprojectlayer",
    {
        'INPUT': gan_layer,
        'TARGET_CRS': QgsCoordinateReferenceSystem('EPSG:25830'),
        'OUTPUT': 'memory:gan_utm'
    }
)['OUTPUT']


## 4. Function to calculate municipal area in hectares

In [ ]:
def calcular_area_ha(layer):
    """
    Calculates the area of each municipality in hectares (ha)
    and saves it in a field called 'ha'
    """
    with edit(layer):

        # Create the 'ha' field if it doesn't exist
        if layer.fields().indexFromName('ha') == -1:
            layer.addAttribute(QgsField('ha', QVariant.Double))

        idx_ha = layer.fields().indexFromName('ha')

        # Calculate area for each feature
        for f in layer.getFeatures():
            area_ha = f.geometry().area() / 10000  # m² → hectares
            layer.changeAttributeValue(f.id(), idx_ha, area_ha)


In [ ]:
# Calculate municipal area in both reprojected layers
calcular_area_ha(agr_utm)
calcular_area_ha(gan_utm)


## 5. Function to calculate farm density

In [ ]:
def calcular_densidad(layer, campo_explotaciones, campo_densidad):
    """
    Calculates farm density:
    density = number of farms / hectares
    """
    with edit(layer):

        # Create the density field if it doesn't exist
        if layer.fields().indexFromName(campo_densidad) == -1:
            layer.addAttribute(QgsField(campo_densidad, QVariant.Double))

        idx_ha = layer.fields().indexFromName('ha')
        idx_exp = layer.fields().indexFromName(campo_explotaciones)
        idx_dens = layer.fields().indexFromName(campo_densidad)

        for f in layer.getFeatures():
            ha = f[idx_ha]
            exp = f[idx_exp]

            # Avoid division by zero or null values
            dens = exp / ha if ha and ha > 0 else None
            layer.changeAttributeValue(f.id(), idx_dens, dens)


In [ ]:
# Agricultural farm density
calcular_densidad(
    agr_utm,
    campo_explotaciones='n_exp_agr',
    campo_densidad='dens_agr'
)

# Livestock farm density
calcular_densidad(
    gan_utm,
    campo_explotaciones='n_exp_gan',
    campo_densidad='dens_gan'
)


## 6. Copy results to original layers (EPSG:4258)

In [ ]:
def copiar_campos(origen, destino, campos):
    """
    Copies values from a temporary reprojected layer
    to the original layer, field by field
    """
    with edit(destino):

        # Create fields in the original layer if they don't exist
        for campo in campos:
            if destino.fields().indexFromName(campo) == -1:
                destino.addAttribute(QgsField(campo, QVariant.Double))

        # Copy values feature by feature
        for f_origen, f_destino in zip(origen.getFeatures(), destino.getFeatures()):
            for campo in campos:
                idx = destino.fields().indexFromName(campo)
                destino.changeAttributeValue(
                    f_destino.id(),
                    idx,
                    f_origen[campo]
                )


In [ ]:
# Copy area and density to original layers
copiar_campos(agr_utm, agr_layer, ['ha', 'dens_agr'])
copiar_campos(gan_utm, gan_layer, ['ha', 'dens_gan'])
